In [11]:
from openai import OpenAI
import os
import math
import base64
from pydub import AudioSegment

from dotenv import load_dotenv

load_dotenv('../.env', override=True)
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

In [12]:
from moviepy.editor import VideoFileClip

def convert_mp4_to_mp3(mp4_file_path, mp3_file_path):
    video_clip = VideoFileClip(mp4_file_path)
    audio_clip = video_clip.audio
    audio_clip.write_audiofile(mp3_file_path)

convert_mp4_to_mp3("../demo-videos/ted-x.mp4", "../temp/ted-x.mp3")

MoviePy - Writing audio in ../temp/ted-x.mp3


MoviePy - Done.


In [5]:
chunk_size_bytes = 20 * 1024 * 1024
file_path = "../temp/ted-x.mp3"
output_folder = "chunks"
num_chunks = -1

In [6]:
def chunk_file_by_size(file_name, chunk_size_bytes, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    
    with open(file_name, 'rb') as file:
        file_size = os.path.getsize(file_name)
        num_chunks = (file_size + chunk_size_bytes -  1) // chunk_size_bytes
        for i in range(num_chunks):
            chunk_file_name = os.path.join(output_folder, f"{os.path.splitext(os.path.basename(file_name))[0]}_chunk{i}.mp3")
            chunk = file.read(chunk_size_bytes)
            with open(chunk_file_name, 'wb') as chunk_file:
                chunk_file.write(chunk)

    return num_chunks

In [7]:
num_chunks = chunk_file_by_size(file_path, chunk_size_bytes, output_folder)

In [8]:
def transcribe_chunk(chunk_file):
    with open(chunk_file, "rb") as audio_file:
        transcription = client.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file,
            response_format="verbose_json",
            timestamp_granularities=["word"]
        )
    return transcription

In [9]:
def get_transcription(file_path, num_chunks):
    transcriptions = []
    
    for i in range(num_chunks):
        chunk_file_name = f"{os.path.splitext(os.path.basename(file_path))[0]}_chunk{i}.mp3"
        
        path = os.path.join(os.getcwd(), output_folder, chunk_file_name)
        transcription = transcribe_chunk(path)
        os.remove(path)

        transcriptions.append(transcription)

    return transcriptions

In [13]:
import time
start_time = time.perf_counter()

transcriptions = get_transcription(file_path, num_chunks)

end_time = time.perf_counter()

time_spent = end_time - start_time
print(f"The line of code took {time_spent:.6f} seconds to execute.")

The line of code took 42.652300 seconds to execute.


In [14]:
print(len(transcriptions[0].text.split(" ")))

2108


In [15]:
import tiktoken
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string(transcriptions[0].text, "cl100k_base")


KeyboardInterrupt: 

In [187]:
# transcriptions[0].text

In [16]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo-0125",
  # response_format={ "type": "json_object" },
  messages=[
    {"role": "system", "content": "You are a helpful text summarizer assistant."},
    {"role": "user", "content": "Summarize this in about 30 lines: " + transcriptions[0].text}
  ]
)
print(response.choices[0].message.content)

A few years ago, the narrator met a student who had recently returned from a Vietnamese monastery and learned a Zen lesson about the meaning of work. This experience led the narrator, now a management consultant, to reflect on the significance of purpose and impact in today's world. Despite the increasing focus on purpose in society, many people still feel that their work is meaningless, leading to disengagement and reduced productivity. Traditional approaches to finding one's purpose, such as through workshops and books, may not be universally effective as they're based on assumptions that don't necessarily hold true.

To address this challenge, the narrator collaborates with a friend from Walmart to develop a more practical approach to understanding purpose. They emphasize the importance of self-impact fit, which involves aligning personal preferences, skills, and societal demands to find fulfilling work. Impact and discovery are key components of purposeful work, and individuals sho